Импорт библиотек

In [1]:
import pandas as pd
import pandahouse as ph
import seaborn as sns
import numpy as np
import datetime
import io
import telegram
import matplotlib.pyplot as plt


In [2]:
# настраиваем бота
bot = telegram.Bot(token='########')
chat_id = ######

отправляем запрос в базу данных

In [4]:
connection = {
    'host': '######',
    'password': '#####',
    'user': '######',
    'database': '#####'
}

In [5]:
query = '''
SELECT *
FROM
  (SELECT *
   FROM
     (SELECT toDate(time) AS day,
             uniqExact(user_id) AS DAU,
             countIf(action='like') AS likes,
             countIf(action='view') AS views,
             round(likes/views*100, 2) AS CTR
      FROM simulator.feed_actions
      WHERE day > today()-7
        AND day < today()
      GROUP BY day
      ORDER BY day) AS posts
   FULL JOIN
     (SELECT toDate(T2.t) AS day,
             round(T2.users/T1.MAU*100, 2) AS Stickiness
      FROM
        (SELECT date_,
                uniqExact(user_id) AS MAU
         FROM
           (SELECT date_ + n AS date_,
                   user_id
            FROM
              (SELECT user_id,
                      toDate(time) AS date_
               FROM simulator.feed_actions) array
            JOIN range(30) AS n)
         WHERE date_ <= today()
         GROUP BY date_) AS T1
      LEFT JOIN
        (SELECT toStartOfDay(toDate(time)) AS t,
                count(DISTINCT user_id) AS users
         FROM simulator.feed_actions
         GROUP BY t) AS T2 ON T2.t = T1.date_
      WHERE day > today()-7
        AND day < today()) AS Stickiness USING day) AS post
FULL JOIN
  (SELECT *
   FROM
     (SELECT toDate(time) AS day,
             uniqExact(user_id) AS mess_DAU
      FROM simulator.message_actions
      WHERE day > today()-7
        AND day < today()
      GROUP BY day) AS mess1
   FULL JOIN
     (SELECT toDate(day) AS day,
             sum(unique_chats) AS unique_chats
      FROM
        (SELECT user_id,
                count(DISTINCT reciever_id) AS unique_chats,
                toStartOfDay(time) AS day
         FROM simulator.message_actions
         WHERE day > today()-7
            AND day < today()   
         GROUP BY user_id,
                  day)
      GROUP BY day
      ORDER BY day) AS mess2 USING day) AS mess USING day
      '''

In [11]:
df = ph.read_clickhouse(query, connection=connection)

In [7]:
# собираем данные за вчера
YD_DAU = round(df.DAU.iloc[-1] / 1000,2)
YD_Stickiness = round(df.Stickiness.iloc[-1])
YD_views = round(df.views.iloc[-1] / 1000,2)
YD_likes = round(df.likes.iloc[-1] / 1000,2)
YD_CTR = round(df.CTR.iloc[-1])


In [8]:
# рисуем графики
sns.set(rc={'figure.figsize':(15.00,20.00)})
fig, axs = plt.subplots(5,1)
sns.lineplot(x='day', y='DAU', data=df, ax=axs[0])
sns.lineplot(x='day', y='Stickiness', data=df, ax=axs[1])
sns.lineplot(x='day',y='views', data=df, ax=axs[2])
sns.lineplot(x='day', y='likes', data=df, ax=axs[3])
sns.lineplot(x='day',y='CTR', data=df, ax=axs[4])
post_plot = io.BytesIO()
plt.tight_layout()
plt.savefig(post_plot)
post_plot.name = 'post_plot.png'
post_plot.seek(0)
plt.close()

# отправляем изображение графика и подпись
bot.sendPhoto(chat_id=chat_id, photo=post_plot, caption=f'''Отчеты по Ленте

За вчера 
DAU: {YD_DAU}k
Sticky factor: {YD_Stickiness}%
Просмотры: {YD_views}k
Лайки: {YD_likes}k
CTR: {YD_CTR}%:''')

In [9]:
# собираем данные за вчера
YD_mess_DAU = round(df.mess_DAU.iloc[-1] / 1000,2)
YD_mess_unique = round(df.unique_chats.iloc[-1] / 1000,2)

In [10]:
# рисуем графики
sns.set(rc={'figure.figsize':(7.50,10.00)})
fig, axs = plt.subplots(2,1)
sns.lineplot(x='day', y='mess_DAU', data=df, ax=axs[0])
sns.lineplot(x='day', y='unique_chats', data=df, ax=axs[1])
mess_plot = io.BytesIO()
plt.tight_layout()
plt.savefig(mess_plot)
mess_plot.name = 'mess_plot.png'
mess_plot.seek(0)
plt.close()

# отправляем изображение графика и подпись
bot.sendPhoto(chat_id=chat_id, photo=mess_plot, caption=f'''Отчеты по Мессенджеру


За вчера 
DAU: {YD_mess_DAU}k
Уникальных чатов: {YD_mess_unique}k''')